# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# Import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# Getting data
orders = pd.read_csv('orders.csv')
display(orders.head(), orders.shape)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


(397924, 14)

In [3]:
#Deleting unnecessary row 'Unnamed:0'
orders = orders.drop(['Unnamed: 0'], axis=1)

In [4]:
# Checking for null values
orders.isna().sum()

InvoiceNo       0
StockCode       0
year            0
month           0
day             0
hour            0
Description     0
Quantity        0
InvoiceDate     0
UnitPrice       0
CustomerID      0
Country         0
amount_spent    0
dtype: int64

In [5]:
#Checking column types

orders.dtypes

InvoiceNo         int64
StockCode        object
year              int64
month             int64
day               int64
hour              int64
Description      object
Quantity          int64
InvoiceDate      object
UnitPrice       float64
CustomerID        int64
Country          object
amount_spent    float64
dtype: object

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

#### Sub Problem 1: How to aggregate the amount_spent for unique customers?

In [6]:
# Group by' Customer_ID', 'Country' and aggregate sum of'amount_spent'

cust_tamount = orders.groupby(['CustomerID', 'Country']).agg({'amount_spent':sum})

display(cust_tamount.head(), cust_tamount.shape)


,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12350,Norway,334.40


(4347, 1)

#### Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?

In [7]:
# Getting data from any quantile

quantiles = cust_tamount['amount_spent'].quantile([0.75,0.95])

quantiles

0.75    1661.195
0.95    5774.278
Name: amount_spent, dtype: float64

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

In [8]:
# Labels
labels=['General Customers','Preferred Customers','VIP Customers']

# Classificating customers according to their total amount quantile
cust_tamount['customer_type'] = pd.cut(cust_tamount['amount_spent'], bins=[-np.inf, quantiles[0.75], quantiles[0.95], np.inf], labels=labels)

cust_tamount.head()

,,amount_spent,customer_type
CustomerID,Country,,
12346,United Kingdom,77183.60,VIP Customers
12347,Iceland,4310.00,Preferred Customers
12348,Finland,1797.24,Preferred Customers
12349,Italy,1757.55,Preferred Customers
12350,Norway,334.40,General Customers


In [9]:
# Adding columns 'vip_count' and 'preferred_count' to sum later

cust_tamount['vip_count'] = cust_tamount['customer_type']== 'VIP Customers'
cust_tamount['preferred_count'] = cust_tamount['customer_type']== 'Preferred Customers'

cust_tamount.head()

,,amount_spent,customer_type,vip_count,preferred_count
CustomerID,Country,,,,
12346,United Kingdom,77183.60,VIP Customers,True,False
12347,Iceland,4310.00,Preferred Customers,False,True
12348,Finland,1797.24,Preferred Customers,False,True
12349,Italy,1757.55,Preferred Customers,False,True
12350,Norway,334.40,General Customers,False,False


In [10]:
# VIP and preferred total counts

display('Total VIP customers: ' , cust_tamount['vip_count'].sum())

display('Total PREFERRED customers: ' , cust_tamount['preferred_count'].sum())

'Total VIP customers: '

218

'Total PREFERRED customers: '

869

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [11]:
# "Vipest" country = group by 'country' and sum 'vip_count'

vipest_country = cust_tamount.groupby(['Country'])['vip_count'].sum().sort_values(ascending=False).head(1)

vipest_country

# cust_tamount.groupby(['Country']).agg({'vip_count':sum})

Country
United Kingdom    178
Name: vip_count, dtype: int64

In [12]:
# Showing top 10 countries with most VIP customers

top_10_vip = cust_tamount.groupby(['Country'])['vip_count'].sum().sort_values(ascending=False).head(10)

top_10_vip


Country
United Kingdom    178
Germany            11
France              9
Switzerland         3
Spain               2
Japan               2
Portugal            2
EIRE                2
Netherlands         1
Singapore           1
Name: vip_count, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [13]:
# Adding column 'VipPre' (addition of VIP and Preferred customers)

cust_tamount['VipPre'] = (cust_tamount['customer_type']== 'VIP Customers')+(cust_tamount['customer_type']== 'Preferred Customers')

cust_tamount.head()

,,amount_spent,customer_type,vip_count,preferred_count,VipPre
CustomerID,Country,,,,,
12346,United Kingdom,77183.60,VIP Customers,True,False,True
12347,Iceland,4310.00,Preferred Customers,False,True,True
12348,Finland,1797.24,Preferred Customers,False,True,True
12349,Italy,1757.55,Preferred Customers,False,True,True
12350,Norway,334.40,General Customers,False,False,False


In [14]:
# "VipPrest" country = group by 'country' and sum 'VipPre'

vipprest_country = cust_tamount.groupby(['Country'])['VipPre'].sum().sort_values(ascending=False).head(1)

vipprest_country

Country
United Kingdom    934
Name: VipPre, dtype: int64

In [15]:
# Showing top 10 countries with most VIP customers
top_10_VipPre = cust_tamount.groupby(['Country'])['VipPre'].sum().sort_values(ascending=False).head(10)

top_10_VipPre


Country
United Kingdom    934
Germany            39
France             29
Belgium            11
Switzerland         9
Spain               7
Portugal            7
Norway              7
Italy               5
Finland             5
Name: VipPre, dtype: int64